# README

The ultimate aim of this notebook is to construct a **naive** *implied order book* for SPX options (options on the S&P500). We will explain what this means in more detail. The data comes from the Chicago Board Options Exchange (CBOE). The data is delayed (hence why it's free!). Here's the link to the page where the data comes from. https://www.cboe.com/delayed_quotes/spx/quote_table

When we make a call to the API we get back a table containing all option contracts on the S&P500 (one row per contract), with some info about the contract:
* The ID tells us the expiry, whether it's a put or a call and the strike price.
* The current bid and ask prices, as well as the bid and ask sizes
* Implied Vol (need to work out whether this is from bid, ask, mid or something else - may need to write my own Black-Scholes function for this)
* Open Interest (total number of open contracts)
* Volume
* The Greeks - delta, gamma, vega, theta, rho, theo
* Some metrics about how the price has changed
* Last traded time and last traded price. This can help to show liquidity.

What this dataset doesn't tell you... EXPLAIN WHY IT'S NAIVE

# Imports

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

# Fetch Data

In [2]:
# Symbol for SPX options
symbol = '_SPX'

# Start session
s = requests.Session()

# Make API call and check status code
response = s.get(f'https://cdn.cboe.com/api/global/delayed_quotes/options/{symbol}.json')
if response.status_code != 200:
    raise Exception(f"Unable to fetch data. Status code is {response.status_code}")

# Get data as json
data = json.loads(response.content)

# Close session
s.close()

In [3]:
# Get timestamp
timestamp = datetime.strptime(data['timestamp'], "%Y-%m-%d %H:%M:%S")

# Get options data
df = pd.DataFrame(
    data=data['data']['options']
)

# Add timestamp column
df['timestamp'] = timestamp

# Convert last trade time to datetime - TODO: Get this to work
# df['last_trade_time'] = df['last_trade_time'].str.replace("T", " ")
# df['last_trade_time'] = pd.to_datetime('last_trade_time', format="%Y-%m-%d %H:%M:%S")


In [4]:
df

,option,bid,bid_size,ask,ask_size,iv,open_interest,volume,delta,gamma,...,change,open,high,low,tick,last_trade_price,last_trade_time,percent_change,prev_day_close,timestamp
0,SPX241115C00200000,5553.8,5.0,5571.80,5.0,7.7264,16.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,down,5655.41,2024-10-18T12:12:30,0.0,5513.449951,2024-11-05 16:30:29
1,SPX241115P00200000,0.0,0.0,0.05,114.0,5.5879,116.0,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,no_change,0.00,None,0.0,0.025000,2024-11-05 16:30:29
2,SPX241115C00400000,5354.1,5.0,5372.10,5.0,6.0068,1.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,down,5428.84,2024-10-21T11:17:52,0.0,5313.849854,2024-11-05 16:30:29
3,SPX241115P00400000,0.0,0.0,0.05,79.0,4.3956,972.0,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,no_change,0.05,2024-10-31T15:47:22,0.0,0.025000,2024-11-05 16:30:29
4,SPX241115C00600000,5154.4,5.0,5172.50,5.0,5.0668,0.0,0.0,1.0000,0.0000,...,0.0,0.0,0.0,0.0,no_change,0.00,None,0.0,5114.300049,2024-11-05 16:30:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24569,SPXW250930P07400000,1370.8,1.0,1403.70,1.0,0.1169,0.0,0.0,-0.9749,0.0001,...,0.0,0.0,0.0,0.0,no_change,0.00,None,0.0,1438.099976,2024-11-05 16:30:29
24570,SPXW250930C07600000,4.2,159.0,4.60,63.0,0.1204,71.0,0.0,0.0194,0.0001,...,0.0,0.0,0.0,0.0,up,5.80,2024-10-30T10:24:52,0.0,4.100000,2024-11-05 16:30:29
24571,SPXW250930P07600000,1560.3,1.0,1593.10,1.0,0.1199,1.0,0.0,-0.9860,0.0001,...,0.0,0.0,0.0,0.0,down,1522.36,2024-10-11T10:17:15,0.0,1627.900024,2024-11-05 16:30:29
24572,SPXW250930C07800000,2.7,171.0,3.10,63.0,0.1240,11.0,0.0,0.0130,0.0000,...,0.0,0.0,0.0,0.0,down,3.10,2024-10-31T12:38:35,0.0,2.725000,2024-11-05 16:30:29


# Data Exploration